In [1]:
using Distributions
using CSV
using DataFrames
using Random

In [2]:
Threads.nthreads()

8

# simulation function

In [3]:
include("main_dist.jl")

┌ Info: Precompiling LPM [top-level]
└ @ Base loading.jl:1662


In [4]:
function run_sim_locally(args)
    # sadly our simulation is not thread-safe so we have to run it via its own julia process
    res = readchomp(`julia main_dist.jl $(split(args))`)
    
    parse(Float64, res)
end

# this works when using ThreadSafeDict for memoization but is sadly much, much slower than running externally
function run_sim_threaded(args)
    distance(split(args))
end

run_sim_threaded (generic function with 1 method)

In [5]:
function simulate(param_values)
    @assert length(names) == length(param_values)
    
    # TODO seeding
    args = "--datadir LPM/data -P \"\""
    
    for (name, value) in zip(names, param_values)
        args *= " --$name $value" 
    end

    run_sim_locally(args)
    #run_sim_threaded(args)
end

simulate (generic function with 1 method)

# config

In [6]:
function readParamConfig(fname)
    pconf = CSV.read(fname, DataFrame)
    
    v_min = pconf[!, :Min]
    v_max = pconf[!, :Max]
    
    priors = [Uniform(mi, ma) for (mi, ma) in zip(v_min, v_max)]
    
    String.(pconf[!, :Parameter]), priors
end
    

readParamConfig (generic function with 1 method)

In [7]:
const names, priors = readParamConfig("params/parameters.tsv");

# ABC

In [8]:
include("abc_hpc/abc/ownabc.jl")

abc (generic function with 1 method)

# execute

In [9]:
noise = [0.05 for p in priors];

In [10]:
particles = Particle[]
remv = PropRemover(0.5)
creat = OwnCreator(true, 1.0, priors, noise)

OwnCreator{Vector{Uniform{Float64}}}(true, 1.0, Uniform{Float64}[Uniform{Float64}(a=0.5, b=1.0), Uniform{Float64}(a=0.5, b=1.0), Uniform{Float64}(a=0.0, b=1.0), Uniform{Float64}(a=0.0, b=2.0), Uniform{Float64}(a=0.0, b=4.0), Uniform{Float64}(a=0.0, b=1.0), Uniform{Float64}(a=0.5, b=1.0), Uniform{Float64}(a=0.0, b=1.0), Uniform{Float64}(a=0.0, b=4.0), Uniform{Float64}(a=0.5, b=1.0), Uniform{Float64}(a=1.0, b=1.2), Uniform{Float64}(a=0.0, b=2.0), Uniform{Float64}(a=10.0, b=100.0), Uniform{Float64}(a=0.0, b=10.0), Uniform{Float64}(a=1.0, b=20.0), Uniform{Float64}(a=0.5, b=1.0), Uniform{Float64}(a=0.05, b=0.2)], [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05])

In [11]:
@time abc_iter!(particles, simulate, 200, remv, creat, verbose=true, parallel = true)

simulating 200 new particles...
3693.119771 seconds (3.68 M allocations: 229.011 MiB, 0.00% gc time, 0.03% compilation time)


200-element Vector{Particle}:
 Particle([0.7905989877380911, 0.7864196270462914, 0.6229991764060937, 1.5715746117820208, 2.611324007780377, 0.8760123790499788, 0.6478074605126627, 0.5376608784172936, 0.31475078494773, 0.9599901903293269, 1.0572298288517188, 1.817428531923197, 24.845914922322404, 3.0084616685734478, 7.827452624657704, 0.9613355677201774, 0.06147907267104623], 0.30211769297969626)
 Particle([0.88909866996229, 0.8153516191329961, 0.8262264795759635, 1.1405278245061485, 2.390892504447173, 0.1626636985529989, 0.8009547609087728, 0.9949651777040871, 3.4004427101952004, 0.9348088951407673, 1.0689277827395767, 1.3318227627726245, 18.209081394284265, 4.184553479797551, 7.65087323864549, 0.6049509712293453, 0.15173336933273657], 0.28135289545909453)
 Particle([0.5539215550957524, 0.5440574681431501, 0.7763963872374479, 0.12446768046580847, 2.10341206790091, 0.8578102012976528, 0.6550041840099994, 0.007560873504934729, 3.0479084789664457, 0.9311753020847244, 1.026914433714355, 0.

In [ ]:
#abc(priors, dist, 200, 0.5, noise, 10, verbose=true, scale_noise=true, parallel=true)
#abc(priors, dist, 4, 0.5, noise, 2, verbose=true, scale_noise=true, parallel=true)
for i in 1:30
    abc_iter!(particles, dist, 200, remv, creat, verbose=true, parallel = true)
    #sort!(particles, by=p->p.dist)
    #push!(meds, particles[end ÷ 2].dist)
    #for i in 1:4
    #    push!(devs[i], std(map(p->p.params[i], particles)))
    #end
end

In [12]:
abc_result = ans

200-element Vector{Particle}:
 Particle([0.7905989877380911, 0.7864196270462914, 0.6229991764060937, 1.5715746117820208, 2.611324007780377, 0.8760123790499788, 0.6478074605126627, 0.5376608784172936, 0.31475078494773, 0.9599901903293269, 1.0572298288517188, 1.817428531923197, 24.845914922322404, 3.0084616685734478, 7.827452624657704, 0.9613355677201774, 0.06147907267104623], 0.30211769297969626)
 Particle([0.88909866996229, 0.8153516191329961, 0.8262264795759635, 1.1405278245061485, 2.390892504447173, 0.1626636985529989, 0.8009547609087728, 0.9949651777040871, 3.4004427101952004, 0.9348088951407673, 1.0689277827395767, 1.3318227627726245, 18.209081394284265, 4.184553479797551, 7.65087323864549, 0.6049509712293453, 0.15173336933273657], 0.28135289545909453)
 Particle([0.5539215550957524, 0.5440574681431501, 0.7763963872374479, 0.12446768046580847, 2.10341206790091, 0.8578102012976528, 0.6550041840099994, 0.007560873504934729, 3.0479084789664457, 0.9311753020847244, 1.026914433714355, 0.

In [13]:
sorted = sort(abc_result, by=p->p.dist);

In [17]:
sorted[1]

Particle([0.899663262579349, 0.8834091193514159, 0.8718376245803687, 1.949735928405904, 0.9391998165833955, 0.09939176484817325, 0.9965345047566001, 0.7201737134951807, 3.9688581450020637, 0.6328159003923108, 1.160561121381467, 0.19746808044443775, 19.479684402452968, 3.826821831225761, 6.401365620052932, 0.5663546127144115, 0.16136506127987205], 0.263272864019611)

In [14]:
real_p = abc_result[1:end-200]
real_p2 = abc_result[end-199:end]

200-element Vector{Particle}:
 Particle([0.7905989877380911, 0.7864196270462914, 0.6229991764060937, 1.5715746117820208, 2.611324007780377, 0.8760123790499788, 0.6478074605126627, 0.5376608784172936, 0.31475078494773, 0.9599901903293269, 1.0572298288517188, 1.817428531923197, 24.845914922322404, 3.0084616685734478, 7.827452624657704, 0.9613355677201774, 0.06147907267104623], 0.30211769297969626)
 Particle([0.88909866996229, 0.8153516191329961, 0.8262264795759635, 1.1405278245061485, 2.390892504447173, 0.1626636985529989, 0.8009547609087728, 0.9949651777040871, 3.4004427101952004, 0.9348088951407673, 1.0689277827395767, 1.3318227627726245, 18.209081394284265, 4.184553479797551, 7.65087323864549, 0.6049509712293453, 0.15173336933273657], 0.28135289545909453)
 Particle([0.5539215550957524, 0.5440574681431501, 0.7763963872374479, 0.12446768046580847, 2.10341206790091, 0.8578102012976528, 0.6550041840099994, 0.007560873504934729, 3.0479084789664457, 0.9311753020847244, 1.026914433714355, 0.

In [ ]:
using Plots

In [ ]:
histogram(map(x->x.params[1], sorted[1:50]), bins=20)

In [ ]:
histogram(map(x->x.params[2], sorted[1:50]), bins=20)

In [ ]:
histogram(map(x->x.params[3], sorted[1:50]), bins=20)

In [ ]:
plotly()
scatter(map(x->1.0-x.params[1], real_p), map(x->x.params[2], real_p), map(x->x.params[3], real_p), 
    xlabel="p_keep", ylabel="p_info", zlabel="p_transfer")

In [ ]:
histogram(map(x->x.params[4], sorted[1:50]), bins=20)

In [ ]:
histogram(map(x->x.dist, sorted[1:100]))